In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# parity check matrix H
H = np.array([[1, 1, 1, 1, 0, 0],
              [0, 0, 1, 1, 0, 1],
              [1, 0, 0, 1, 1, 0]])

In [3]:
H.shape

(3, 6)

https://en.wikipedia.org/wiki/Low-density_parity-check_code

In [4]:
def generate_H_hat(H):
    
    rows, cols = H.shape
    N, K = cols, cols - rows
    M = N - K
    H_hat = H.copy()
    
    for i in range(M):
        j = 1
        while H_hat[i][K + i] != 1:
            flag = (i + j) % M
            sum_ = H_hat[i] + H_hat[flag]
            H_hat[i] = sum_ % 2
            j += 1
            
        for k in range(M):
            if k == i:
                continue
            if H_hat[k][K + i] == 1:
                sum_ = H_hat[i] + H_hat[k]
                H_hat[k] = sum_ % 2
    return H_hat

In [5]:
def generate_G(H):
    
    rows, cols = H.shape
    N, K = cols, cols - rows
    M = N - K
    H_hat = generate_H_hat(H)
    
    I = np.eye(K, dtype=int)
    P = H_hat[:, :K]
    G = np.vstack((I, P))

    return G

In [6]:
H_hat = generate_H_hat(H)
G = generate_G(H)

print("H_hat:")
print(H_hat)
print("\nG:")
print(G)

H_hat:
[[1 1 1 1 0 0]
 [0 1 1 0 1 0]
 [1 1 0 0 0 1]]

G:
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 1]
 [0 1 1]
 [1 1 0]]


In [7]:
# read files
H1 = np.loadtxt(fname="H1.txt", dtype=int)
y1 = np.loadtxt(fname="y1.txt", dtype=int)

In [8]:
H1.shape

(750, 1000)

In [9]:
y1.shape

(1000,)

In [10]:
# read files
H1 = np.loadtxt(fname="H1.txt", dtype=int)
y1 = np.loadtxt(fname="y1.txt", dtype=int)

In [11]:
def decoder(H, y, p=0.1, max_itera_times=20):

    rows, cols = H.shape
    N, K = cols, cols - rows
    M = N - K
    nums_y = len(y)
    
    p_x1 = np.zeros(shape=nums_y)
    p_x0 = np.zeros(shape=nums_y)
    loglik = np.zeros(shape=(rows, cols))
    bits = []
    itera_times = 0
    
    for i in range(nums_y):
        p_x1[i] = p ** ((y[i] + 1) % 2) * (1 - p) ** ((y[i] + 2) % 2)
        p_x0[i] = p ** y[i] * (1 - p) ** ((y[i] + 1) % 2)
#     print(p_x1)
#     print(p_x0)
    
    # log likelihood will be faster
    for row in range(rows):
        bits.append(np.where(H[row]==1)[0])
        for col in range(cols):
            loglik[row, col] = np.log(p_x0[col] / p_x1[col])
    
#     print(loglik)

    while True:
        itera_times += 1
        
        loop_continued = False
        loglik_factorized = np.zeros(shape=(rows, cols))
        pred = np.zeros(shape=cols)
        
        for row in range(rows):
            column = np.zeros(shape=cols)
            prod = loglik[row, bits[row]]
            
            for i in range(len(prod)):
                prod_ = np.prod(np.tanh(np.hstack((prod[0:i], prod[i + 1:len(prod)])) / 2))
#                 prod_ = 1
#                 for j in temp:
#                     prod_ *= j
                column[bits[row][i]] = np.log((1 + prod_) / (1 - prod_))
    
            loglik_factorized[row, :] = column
        
#         print(loglik_factorized)
        posterior = loglik_factorized.sum(axis=0)
        for i in range(len(posterior)):
            posterior[i] += np.log(p_x0[i] / p_x1[i])
#         print(posterior)   
        for i in range(len(pred)):
            pred[i] = 0 if posterior[i] > 0 else 1
#         print(len(posterior))
#         print(len(pred))
        
        temp = np.matmul(H, pred.T) % 2
        for element in temp:
            if element != 0:
                loop_continued = True
                break
        if loop_continued and itera_times < max_itera_times:
            for row in range(rows):
                for col in range(cols):
                    loglik[row, col] = posterior[col] - loglik_factorized[row, col]
        else:
            break
    if itera_times <= 20:        
        return pred, itera_times, 0
    else:
        return pred, itera_times, -1


In [12]:
def translator(decode_vec):
    message = []
    for i in range(31):
        char_ = ''
        for j in range(8):
            char_ += (decode_vec[i * 8 + j].astype(int)).astype(str)
#         print(char)
        message.append(chr(int(char_, 2)))
    return message

In [13]:
decoded_vector, itera_times, return_code = decoder(H1, y1)

print("It takes {} iterations.".format(itera_times))

It takes 8 iterations.


In [14]:
print("Decoded Vector:")
print(decoded_vector)

Decoded Vector:
[0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1.
 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0

In [15]:
print("Return code: {}.".format(return_code))

Return code: 0.


In [16]:
message = translator(decoded_vector)
print(''.join(message))

Happy Holidays! Dmitry&David :)
